In [2]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation
from tensorflow.keras.optimizers import RMSprop

filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
text=open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [5]:
text = text[300000:800000]#part of data

In [6]:
chars = sorted(list(set(text))) #filter out all of unique charchters and sorting them like abcd df
char_indices = dict((c, i) for i, c in enumerate(chars)) #enumarte them till the max convert them to numerical char is key index is value
indices_char = dict((i, c) for i, c in enumerate(chars)) #from numerical to char again

In [8]:
SEQ_LENGTH = 40 #40 char predict next char
STEP_SIZE = 3 #how many char are going to shift
sentences = [] #features
next_chars = [] #target

In [9]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE): #loops from beinign of text to last part of text
    sentences.append(text[i: i + SEQ_LENGTH]) #5 0 to 4
    next_chars.append(text[i + SEQ_LENGTH]) #5 is next char

In [12]:
#convert data to numpy array
x=np.zeros((len(sentences), SEQ_LENGTH, len(chars)), dtype=bool)
y=np.zeros((len(sentences), len(chars)), dtype=bool)

In [13]:
for i , sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1 #sentence num i at position num t and char num whatever this position is set to 1
    y[i, char_indices[next_chars[i]]] = 1 #target data same thing

In [14]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(chars))))
model.add(Dense(len(chars))) #if we have 10 char then it is layer of 10 neurons
model.add(Activation('softmax')) #scale output all values scale to one

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [16]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))

In [17]:
model.fit(x, y, batch_size=256, epochs=4) #batch size how many example we willl put into the network

Epoch 1/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 119s 179ms/step - loss: 2.4701
Epoch 2/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 144s 183ms/step - loss: 1.7755
Epoch 3/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 134s 170ms/step - loss: 1.6083
Epoch 4/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 147s 177ms/step - loss: 1.5274


In [18]:
def sample(preds, temperature=1.0): #temperature predictions of our model and picks one char depend on temp pick is consevative or expermintal
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [19]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length): #produce as many char
        x = np.zeros((1, SEQ_LENGTH, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1 #at certain position the char conert to index

        predictions = model.predict(x, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
    return generated

In [20]:
print(generate_text(300, 0.2))

 the greatest, able to do least,
yet most the fair stand the strength the brother hath strong.

romeo:
at thou say the hands the father that stand
that i seem the marriage that the brother this dear the strength my servest.

clown:
i so that hast the fair the father the strength the stread the strength the strength the chill stand.

king 


In [21]:
print(generate_text(300, 0.6))

'tis a derivative from me to mine,
and of warwick of the sacross that bear the chilt.

romeo:
archang warman a first this sander the most what
i have seem the worst at out.
the brother would side she haster doth a drows of this and counter's both the defence
it the traze is what not i have been in the thing?

warwick:
o'er lie lament the 


In [22]:
print(generate_text(300, 0.8))

 unborn and unbegot,
that lift your vass i may gentlems as stand
with his goodle eccursains him before not,
this armion.

henry bolingboung:
what welt they shall not perour again,
berouden fage of thron and the stranger
the brothe show in little offeress
to fain that fair countres, myselfood
as the stres and sting storld their serrecty
th
